## Down-Sampling, Up-Sampling & Class-Weighting

In [ ]:
from pandas import Series, DataFrame 
import pandas as pd
%pylab inline
import warnings
warnings.filterwarnings('ignore')

In [140]:
##function Definitions
def dnsmpl(xtrain,ytrain,p):
    train=xtrain
    train['y']=ytrain[0]
    positive=train['y'].sum(axis =0)
    negative=int(train['y'].sum(axis =0)*(1-p)/p)
    train_0=train[train['y']==0]
    train_1=train[train['y']==1]
    train_0_down_sampled = train_0.sample( negative)
    train_final=train_1.append(train_0_down_sampled)
    return(train_final)
    
    
def upsmpl(xtrain,ytrain,p):
    train=xtrain
    train['y']=ytrain[0]
    negative=len(train[train['y']==0])
    positive=train['y'].sum(axis =0)
    replicate=int(((float(negative)+float(positive))*0.1-float(positive))/(1-0.1))
    train_0=train[train['y']==0]
    train_1=train[train['y']==1]
    train_1_up_sampled = train_1.sample(replicate)
    train_final=train.append(train_1_up_sampled,ignore_index=True)
    
    train_final['y'].value_counts()/len(train_final)
    return(train_final)    

In [205]:
##Down sampling

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import scale

scale
lr_results={}
for p in xrange(10,35,5):
    p1=float(p)/float(100)
    f1_score=[]
    for i in xrange(1,6):
        tr_x="vowel_train%s.csv" % i
        tr_y='vowel_tr_label%s.csv'% i
        tst_x="vowel_test%s.csv" % i
        tst_y='vowel_tst_label%s.csv'% i
        train_x = pd.read_csv(tr_x,header=None)
        train_x=DataFrame(scale(train_x))
        train_x=train_x.drop(0,1)
        train_y=pd.read_csv(tr_y,header=None)
        test_x=pd.read_csv(tst_x,header=None)
        test_x=DataFrame(scale(test_x))
        test_x=test_x.drop(0,1)
        test_y=pd.read_csv(tst_y,header=None)
        dwn_train=dnsmpl(train_x,train_y,p1)
        dwn_train_y=dwn_train['y']
        dwn_train_x=dwn_train.drop(['y'],1)
        model = LogisticRegression()
        result = model.fit(dwn_train_x, dwn_train_y)
        prediction_train = model.predict(test_x)
        f1=metrics.f1_score(test_y, prediction_train)
        f1_score.append(f1)
    lr_results[p1]=mean(f1_score)

    

lr_results1=pd.Series(lr_results) 

lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 

lr_results1.order(ascending=False)




Best parameter for under sampling 0.15


0.15    0.817329
0.10    0.813414
0.20    0.764309
0.25    0.723971
0.30    0.697802
dtype: float64

In [200]:
##Up sampling
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr_results={}
for p in xrange(10,35,5):
    p1=float(p)/float(100)
    f1_score_up=[]
    for i in xrange(1,6):
        tr_x="vowel_train%s.csv" % i
        tr_y='vowel_tr_label%s.csv'% i
        tst_x="vowel_test%s.csv" % i
        tst_y='vowel_tst_label%s.csv'% i
        train_x = pd.read_csv(tr_x,header=None)
        train_x=DataFrame(scale(train_x))
        train_x=train_x.drop(0,1)
        train_y=pd.read_csv(tr_y,header=None)
        test_x=pd.read_csv(tst_x,header=None)
        test_x=DataFrame(scale(test_x))
        test_x=test_x.drop(0,1)
        test_y=pd.read_csv(tst_y,header=None)
        up_train=upsmpl(train_x,train_y,p1)
        up_train_y=up_train['y']
        up_train_x=up_train.drop(['y'],1)
        model = LogisticRegression()
        result = model.fit(up_train_x, up_train_y)
        prediction_train = model.predict(test_x)
        f1=metrics.f1_score(test_y, prediction_train)
        f1_score_up.append(f1)
    lr_results[p1]=mean(f1_score_up)

    

lr_results1=pd.Series(lr_results) 

lr_results_best=lr_results1.order(ascending=False).index[0]
print 'Best parameter for under sampling', lr_results_best 

lr_results1.order(ascending=False)



Best parameter for under sampling 0.25


0.25    0.820990
0.10    0.814216
0.30    0.812615
0.20    0.806641
0.15    0.806641
dtype: float64

In [195]:
##ClassWeighting
weight={0:1,1:2}
f1_score_wt=[]
for i in xrange(1,6):
        tr_x="vowel_train%s.csv" % i
        tr_y='vowel_tr_label%s.csv'% i
        tst_x="vowel_test%s.csv" % i
        tst_y='vowel_tst_label%s.csv'% i
        train_x = pd.read_csv(tr_x,header=None)
        train_x=DataFrame(scale(train_x))
        train_x=train_x.drop(0,1)
        train_y=pd.read_csv(tr_y,header=None)
        test_x=pd.read_csv(tst_x,header=None)
        test_x=DataFrame(scale(test_x))
        test_x=test_x.drop(0,1)
        test_y=pd.read_csv(tst_y,header=None)
        model = LogisticRegression(class_weight=weight)
        result = model.fit(train_x, train_y.values)
        prediction_train = model.predict(test_x)
        f1=metrics.f1_score(test_y.values, prediction_train)
        
        f1_score_wt.append(f1)
        
print 'Mean F1 Score',(mean(f1_score_wt))

f1_score_wt
        

Mean F1 Score 0.814906322027


[0.82352941176470595,
 0.81818181818181812,
 0.81081081081081086,
 0.89473684210526316,
 0.72727272727272718]

In [197]:
####Vanilla Logistic Regression

f1_score_wt=[]
for i in xrange(1,6):
        tr_x="vowel_train%s.csv" % i
        tr_y='vowel_tr_label%s.csv'% i
        tst_x="vowel_test%s.csv" % i
        tst_y='vowel_tst_label%s.csv'% i
        train_x = pd.read_csv(tr_x,header=None)
        train_x=DataFrame(scale(train_x))
        train_x=train_x.drop(0,1)
        train_y=pd.read_csv(tr_y,header=None)
        test_x=pd.read_csv(tst_x,header=None)
        test_x=DataFrame(scale(test_x))
        test_x=test_x.drop(0,1)
        test_y=pd.read_csv(tst_y,header=None)
        model = LogisticRegression()
        result = model.fit(train_x, train_y.values)
        prediction_train = model.predict(test_x)
        f1=metrics.f1_score(test_y.values, prediction_train)
        
        f1_score_wt.append(f1)
        
print 'Mean F1 Score',(mean(f1_score_wt))

f1_score_wt



Mean F1 Score 0.806640530905


[0.75,
 0.86486486486486491,
 0.88235294117647056,
 0.84848484848484851,
 0.68750000000000011]